# Recap:

In my last post -- over a year ago -- I showed how to use linear programming to pick a simplified FPL team.

The code is below:

In [1]:
import pulp
import numpy as np

def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

Since the 2020/21 season has just begun, let's see what team we pick for this season:

In [2]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/players_raw.csv"
)
expected_scores = df["total_points"] / 38  # penalises players who played fewer games
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]

decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


/home/joe/forecasting-fantasy-football/.env/lib/python3.6/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 61.82105263157895

First Team:
Ashley Westwood
Nick Pope
Virgil van Dijk
Andrew Robertson
Trent Alexander-Arnold
Kevin De Bruyne*
Anthony Martial
John Lundstram
Danny Ings
Matt Doherty
Raúl Jiménez

Subs:
Dale Stephens
Mathew Ryan
John Egan
Mark Noble


The above model selects the optimal starters, subs and captain from a set of players with perfect score forecasts and using a simplified treatment of substitutions.

This is a decent model of the problem of picking an FPL team from scratch at the beginning of a season. In reality, we have to solve the much more complex, *dynamic* problem of picking a team and planning transfers.

Let's set up a simplified task. We already have a selected team and would like to know which transfers will maximise next week's score. We can represent our selected team in the same way as before, as a vector of binary values.

Our decisions will be which players to transfer in and which to transfer out. I'm going to revert to a starting-eleven-only model for now to make this more clear.

In [3]:
num_players = 100
current_team_indices = np.random.randint(0, num_players, size=11)  # placeholder
clubs = np.random.randint(0, 20, size=100)  # placeholder
positions = np.random.randint(1, 5, size=100)  # placeholder
expected_scores = np.random.uniform(0, 10, size=100)  # placeholder

#current_sub_indices = np.random.randint(0, num_players, size=4)  # placeholder
#current_captain_indices = current_team_indices[0]  # placeholder

# convert to binary representation
current_team_decisions = np.zeros(num_players) 
current_team_decisions[current_team_indices] = 1
# convert to binary representation
#current_sub_decisions = np.zeros(num_players) 
#current_sub_decisions[current_sub_indices] = 1
# convert to binary representation
#current_captain_decisions = np.zeros(num_players) 
#current_captain_decisions[current_captain_indices] = 1

model = pulp.LpProblem("Transfer optimisation", pulp.LpMaximize)

transfer_in_decisions = [
    pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
    for i in range(num_players)
]
transfer_out_decisions = [
    pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
    for i in range(num_players)
]

next_week_team = [
    current_team_decisions[i] + transfer_in_decisions[i] - transfer_out_decisions[i]
    for i in range(num_players)
]

This takes each player and adds or removes him from the team depending on the transfer decisions. This requires a bunch of new constraints.

* Only players in the team can be transferred out
* Only players not in the team can be transferred in
* A player cannot be transferred in and out at the same time
* Players should only be transferred with others in the same position

We can program the first two in implicitly by constraining the `next_week_team` variables to zero or one. For now, instead of explicitly programming the last constraint I'm just going to have `next_week_team` satisfy the formation constraints from our earlier model. Once we include substitutes, this will automatically enforce the last rule.

In [4]:
for i in range(num_players):
    model += next_week_team[i] <= 1
    model += next_week_team[i] >= 0
    model += (transfer_in_decisions[i] + transfer_out_decisions[i]) <= 1
    
# formation constraints
# 1 starting goalkeeper
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 1) == 1

# 3-5 starting defenders
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 2) >= 3
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 2) <= 5

# 3-5 starting midfielders
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 3) >= 3
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 3) <= 5

# 1-3 starting attackers
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 4) >= 1
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 4) <= 3

# club constraint
for club_id in np.unique(clubs):
    model += sum(next_week_team[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

model += sum(next_week_team) == 11  # total team size


We also need to deal with prices. We will make the simplifying assumption that prices do not change over time.

In [5]:
# placeholder budget and prices
budget_now = 0
buy_prices = sell_prices = np.random.uniform(4, 12, size=100)

transfer_in_cost = sum(transfer_in_decisions[i] * buy_prices[i] for i in range(num_players))
transfer_out_cost = sum(transfer_in_decisions[i] * sell_prices[i] for i in range(num_players))

budget_next_week = budget_now + transfer_out_cost - transfer_in_cost
model += budget_next_week >= 0

Finally the objective

In [6]:
# objective function:
model += sum((next_week_team[i]) * expected_scores[i]
             for i in range(num_players)), "Objective"

In [7]:
model.solve()

1

In [8]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(i, buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(i, sell_prices[i], expected_scores[i]))

Transferred in: 2 9.73442993846028 8.864434500678488
Transferred in: 13 11.931103201872991 8.508116925064806
Transferred out: 26 11.134368519334021 1.0242605226468915
Transferred in: 32 10.640630671506848 9.344173611512575
Transferred out: 33 4.499802987498393 0.5179668200896625
Transferred in: 39 9.608999944612519 9.21765106931958
Transferred out: 43 5.5683812824750465 6.368033996230643
Transferred out: 47 7.19727837344396 8.44318314088763
Transferred in: 52 9.383326346556778 8.859627548595167
Transferred out: 54 9.851251974570928 6.770809464800019
Transferred in: 56 5.322948790980421 8.754997931164377
Transferred out: 60 8.57027932845811 0.32058684583684216
Transferred in: 64 5.070708672751263 8.561406207190121
Transferred in: 76 4.207946955311924 8.662469952227257
Transferred out: 90 8.770576797251092 3.9831336314730956
Transferred in: 95 11.956570179557263 8.91433165885844


We forgot add a penalty for making transfers! This is simple, but requires free and non-free transfers to be treated separately to maintain linearity.

Let's wrap this all up into a class `TransferOptimiser`. I've added substitues back in here, and allowed buy and sell prices to be different.

In [9]:
import sys
sys.path.append("..")
from fpl_opt.transfers import TransferOptimiser

In [10]:
num_players = 100
current_squad_indices = np.random.randint(0, num_players, size=15)
clubs = np.random.randint(0, 20, size=100)
positions = np.random.randint(1, 5, size=100)
expected_scores = np.random.uniform(0, 10, size=100)
current_squad_decisions = np.zeros(num_players) 
current_squad_decisions[current_team_indices] = 1
# placeholder budget and prices
budget_now = 0
buy_prices = sell_prices = np.random.uniform(4, 12, size=100)

opt = TransferOptimiser(expected_scores, buy_prices, sell_prices, positions, clubs)

In [11]:
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(current_squad_indices, budget_now, sub_factor=0.2)

Solver status: 1


In [12]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(i, buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(i, sell_prices[i], expected_scores[i]))

Transferred in: 0 4.063162243123512 8.992295878144388
Transferred in: 10 5.532213300265095 9.598219070362006
Transferred out: 24 9.626909366374127 3.6316033386893585
Transferred out: 38 8.03415458862758 2.1046100010414936
Transferred out: 41 5.7146887241131115 0.946000661535843
Transferred in: 50 9.894597049437925 9.86605945476136
Transferred in: 51 11.25680731745625 9.808139720134042
Transferred out: 62 11.945787566289745 3.487565527192309
Transferred out: 64 6.997764236971407 0.835103039518793
Transferred in: 66 7.186332820864807 9.716992969033099


Let's apply this to my current team, again using a very simple player score forecast.

In [13]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/players_raw.csv"
)
expected_scores = df["total_points"] / 38  # penalises players who played fewer games
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]

decisions, captain_decisions, sub_decisions = select_team(expected_scores, prices.values, positions.values, clubs.values)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""), expected_scores[i], prices[i])
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i], expected_scores[i], prices[i])
        player_indices.append(i)

Total expected score = 61.82105263157895

First Team:
Ashley Westwood 3.1052631578947367 5.5
Nick Pope 4.473684210526316 5.5
Virgil van Dijk 4.684210526315789 6.5
Andrew Robertson 4.7631578947368425 7.0
Trent Alexander-Arnold 5.526315789473684 7.5
Kevin De Bruyne* 6.605263157894737 11.5
Anthony Martial 5.2631578947368425 9.0
John Lundstram 3.789473684210526 5.5
Danny Ings 5.2105263157894735 8.5
Matt Doherty 4.394736842105263 6.0
Raúl Jiménez 5.105263157894737 8.5

Subs:
Dale Stephens 1.6578947368421053 4.5
Mathew Ryan 3.5526315789473686 4.5
John Egan 3.5 5.0
Mark Noble 2.763157894736842 5.0


In [14]:
# next week score forecast: start with points-per-game
score_forecast = df["total_points"] / 38
# let's make up a nonsense forecast to add some dynamics -- +1 to Chelsea players
score_forecast.loc[df["team_code"] == 8] += 1
# -1 for Liverpool players
score_forecast.loc[df["team_code"] == 14] -= 1
score_forecast = score_forecast.fillna(0)

In [15]:
opt = TransferOptimiser(score_forecast.values, prices.values, prices.values, positions.values, clubs.values)
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(player_indices, budget_now=0, sub_factor=0.2)

/home/joe/forecasting-fantasy-football/.env/lib/python3.6/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1


In [16]:
for i in range(len(transfer_in_decisions)):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(names[i], prices[i], score_forecast[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(names[i], prices[i], score_forecast[i]))

Transferred in: César Azpilicueta 6.0 4.421052631578947
Transferred out: Virgil van Dijk 6.5 3.6842105263157894


In [17]:
player_indices = []
print()
print("First Team:")
for i in range(len(starters)):
    if starters[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


First Team:
Ashley Westwood
Nick Pope
César Azpilicueta
Andrew Robertson
Trent Alexander-Arnold
Kevin De Bruyne*
Anthony Martial
John Lundstram
Danny Ings
Matt Doherty
Raúl Jiménez

Subs:
Dale Stephens
Mathew Ryan
John Egan
Mark Noble


/home/joe/forecasting-fantasy-football/.env/lib/python3.6/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpAffineExpression'>


In [49]:
HORIZON = 4
multi_score_forecast = pd.DataFrame({"week_{}".format(i): df["total_points"] / 38 for i in range(HORIZON)})
multi_score_forecast.head()


multi_score_forecast.loc[df["team_code"] == 8, "week_0"] += 1
multi_score_forecast.loc[df["team_code"] == 8, "week_1"] += 0
multi_score_forecast.loc[df["team_code"] == 8, "week_2"] -= 1
multi_score_forecast.loc[df["team_code"] == 8, "week_3"] -= 1

multi_score_forecast.loc[df["team_code"] == 14, "week_0"] -= 1
multi_score_forecast.loc[df["team_code"] == 14, "week_1"] += 1
multi_score_forecast.loc[df["team_code"] == 14, "week_2"] += 1
multi_score_forecast.loc[df["team_code"] == 14, "week_3"] -= 1

from fpl_opt.transfers import MultiHorizonTransferOptimiser
opt = MultiHorizonTransferOptimiser(multi_score_forecast.values.T, prices.values, prices.values, positions.values, clubs.values, 4)
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(player_indices, budget_now=0, sub_factor=0.2)

for week in range(len(transfer_in_decisions)):
    print("Week {}".format(week))
    for i in range(len(transfer_in_decisions[week])):
        if transfer_in_decisions[week][i].value() == 1:
            print("Transferred in: {} {} {}".format(names[i], prices[i], multi_score_forecast.values.T[week][i]))
        if transfer_out_decisions[week][i].value() == 1:
            print("Transferred out: {} {} {}".format(names[i], prices[i], multi_score_forecast.values.T[week][i]))

Week 0
Transferred in: James Tarkowski 5.5 3.763157894736842
Transferred out: César Azpilicueta 6.0 3.421052631578947
Week 1
Transferred out: Ashley Westwood 5.5 3.1052631578947367
Transferred in: Jordan Henderson 5.5 4.052631578947368
Week 2
Week 3
Transferred in: Ashley Westwood 5.5 3.1052631578947367
Transferred out: Jordan Henderson 5.5 2.0526315789473686
